In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from generate import generate
from transformers import AutoTokenizer, AutoModel


device = 'cuda'
model = AutoModel.from_pretrained('GSAI-ML/LLaDA-8B-Instruct', trust_remote_code=True, torch_dtype=torch.bfloat16).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained('GSAI-ML/LLaDA-8B-Instruct', trust_remote_code=True)

/home/sagoyal/research/DualDiffusion/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 6/6 [00:00<00:00, 150.11it/s]


In [3]:
gen_length = 64
steps = 32
print('*' * 66)
print(f'**  Answer Length: {gen_length}  |  Sampling Steps: {steps}  **')
print('*' * 66)

conversation_num = 0
user_input = "Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?"

m = [{"role": "user", "content": user_input}]
user_input = tokenizer.apply_chat_template(m, add_generation_prompt=True, tokenize=False)
input_ids = tokenizer(user_input)['input_ids']
input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)

if conversation_num == 0:
    prompt = input_ids
else:
    prompt = torch.cat([prompt, input_ids[:, 1:]], dim=1)

******************************************************************
**  Answer Length: 64  |  Sampling Steps: 32  **
******************************************************************


In [4]:
from generate import generate, generate_per_step

print(prompt)

out = generate_per_step(model, 
    prompt, 
    n=steps, 
    k = 1,
    gen_length=gen_length, 
    block_length=gen_length, 
    temperature=0.0, 
    cfg_scale=0., 
    remasking='low_confidence')

# print("Direct out: ", out)

answer = tokenizer.batch_decode(out[:, prompt.shape[1]:], skip_special_tokens=True)[0]
print(f"Bot's reply: {answer}")

# remove the <EOS>
prompt_out = out[out != 126081].unsqueeze(0)
conversation_num += 1
print('-----------------------------------------------------------------------')

tensor([[126080, 126346,   3840, 126347,    198,    198,  18346,   1166,    341,
          16014,   3141,    352,    259,    782,  24943,    818,   6004,    558,
             16,     15,     15,     13,  50829,    723,   1191,   4555,    300,
            268,   3141,   1285,   3393,     13,   6617,   6393,   6590,    297,
           2697,   1094,    558,     16,     20,    352,    378,   6189,     11,
            301,   1094,  47758,  11553,    409,   1670,    409,   1094,   6393,
             13,   2071,   1670,    778,   3141,   1543,  50829,   1050,    297,
           3397,    268,  24943,     30, 126348, 126346,    598,  10450, 126347,
            198,    198]], device='cuda:0')

Top-10 tokens per position at step 0:

Position 133:
  Rank   Token                          Probability 
  ------------------------------------------------
  1                                     0.259766     ◄ SELECTED
  2       is                            0.202148     
  3      :                      